In [ ]:
!pip install transformers
!pip install sentencepiece

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import torch

model_path = 'adenhaus/mt5-large-stata'
tokenizer = MT5Tokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path)

class RegressionLogitsProcessor(torch.nn.Module):
    def __init__(self, extra_token_id):
        super().__init__()
        self.extra_token_id = extra_token_id

    def __call__(self, input_ids, scores):
        extra_token_logit = scores[:, :, self.extra_token_id]
        return extra_token_logit

def preprocess_inference_input(input_text):
    input_encoded = tokenizer(input_text, return_tensors='pt')
    return input_encoded

tokenizer_config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/866 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
unused_token = "<extra_id_1>"

def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

def do_regression(input_str):
  input_data = preprocess_inference_input(input_str)

  logits_processor = RegressionLogitsProcessor(tokenizer.get_vocab()[unused_token])

  output_sequences = model.generate(
      **input_data,
      max_length=2,  # Generate just the regression token
      do_sample=False,  # Important: Disable sampling for deterministic output
      return_dict_in_generate=True,  # Get the scores directly
      output_scores=True
  )

  # Extract the logit
  unused_token_id = tokenizer.get_vocab()[unused_token]
  regression_logit = output_sequences.scores[0][0][unused_token_id]

  regression_score = sigmoid(regression_logit).item()

  return regression_score

In [ ]:
import pandas as pd

# Read CSV file
df = pd.read_csv('test_stata.csv')

# Initialize lists to store the strings and predictions
input_strings = []
predictions = []
attributable = []

df = df.iloc[:100]

# Create strings and call predict function for each row
for index, row in df.iterrows():
    linearized_text = str(row['linearized_input'])
    output_value = row['output']
    attributable_col = row['attributable']

    # Create a string by concatenating linearized_text, '[output]', and output_value
    input_string = row['full_input']

    # Append the string to the input_strings list
    input_strings.append(input_string)

    # Assuming you have a predict function that takes a string and returns a prediction
    prediction = do_regression(input_string)
    print('index: ' + str(index))
    print('prediction: ' + str(prediction))
    print('reference: ' + str(attributable_col))
    print()

    # Append the prediction to the predictions list
    predictions.append(prediction)

    attributable.append(attributable_col)

    # print(input_string)
    # print(prediction)
    # print()

# Display the results
result_df = pd.DataFrame({'Input String': input_strings, 'prediction': predictions, 'attributable': attributable})
print(result_df)
result_df.to_csv('stata_annotations.csv', index=False)

index: 0
prediction: 0.4977132976055145
reference: 0.0

index: 1
prediction: 0.5066059231758118
reference: 0.0

index: 2
prediction: 0.5084674954414368
reference: 0.0

index: 3
prediction: 0.49275916814804077
reference: 0.0

index: 4
prediction: 0.4966872036457062
reference: 0.0

index: 5
prediction: 0.5079290866851807
reference: 0.0

index: 6
prediction: 0.7068977355957031
reference: 0.0

index: 7
prediction: 0.4999026358127594
reference: 0.0

index: 8
prediction: 0.5007340908050537
reference: 0.0

index: 9
prediction: 0.6610340476036072
reference: 1.0

index: 10
prediction: 0.6944400668144226
reference: 1.0

index: 11
prediction: 0.7048084139823914
reference: 1.0

index: 12
prediction: 0.5546693205833435
reference: 0.0

index: 13
prediction: 0.5130051970481873
reference: 0.0

index: 14
prediction: 0.5104072093963623
reference: 0.0

index: 15
prediction: 0.5982892513275146
reference: 0.0

index: 16
prediction: 0.572989821434021
reference: 0.0

index: 17
prediction: 0.5135657787322998


In [ ]:
import pandas as pd
from scipy.stats import pearsonr

def calculate_pearson_correlation(df, column1, column2):
    # Calculate Pearson correlation coefficient
    correlation_coefficient, _ = pearsonr(df[column1], df[column2])

    print(correlation_coefficient)

# Calculate Pearson correlation coefficient between 'attributable' and 'prediction'
correlation = calculate_pearson_correlation(result_df, 'attributable', 'prediction')

0.680770787003389
